In [1]:
from datascience import *
import numpy as np

## 1.Toy data

For the very first example we are going to create a small toy dataset. In general, until you get a good handle on the functions discussed in this notebook, it is advised that you try your code on a small example like the one bellow. This way you can verify that your code is working correctly! For bigger data sets it might be difficult to evaluate the code's behaviour based on just a handful of rows that Python outputs. 

In [2]:
toy = Table().with_columns("id", np.repeat([1,2,3], 3),
                           "group", make_array("A","B","C","B","B","B","A", "A", "C"),
                           "val_1", np.arange(1,10),
                           "val_2", np.random.normal(0, 1, 9))
toy

id,group,val_1,val_2
1,A,1,-0.535532
1,B,2,-0.347885
1,C,3,0.345067
2,B,4,-0.911886
2,B,5,1.34077
2,B,6,0.0547437
3,A,7,-0.659394
3,A,8,-2.7284
3,C,9,-1.34297


**Question 1.1.** How many of each 'id' do we have? *Hint: use .group()*

In [3]:
toy.group("id")

id,count
1,3
2,3
3,3


**Question 1.2.** How many of each group do we have?

In [4]:
toy.group("group")

group,count
A,3
B,4
C,2


**Question 1.3.** How many observations do we have for each pair id--group?

In [5]:
toy.group(["id", "group"]) # Note the square brackets!

id,group,count
1,A,1
1,B,1
1,C,1
2,B,3
3,A,2
3,C,1


Note that in the example above the combinations id--group that don't have any observations in them (for example 2--A) are omitted.

**Question 1.4.** What is the minimum value of val_2 for each pair id--group?

In [6]:
toy.drop("val_1").group(["id", "group"], min)

id,group,val_2 min
1,A,-0.535532
1,B,-0.347885
1,C,0.345067
2,B,-0.911886
3,A,-2.7284
3,C,-1.34297


**Question 1.5.** How many observations do we have for each pair id--group? *Hint: use .pivot()*

In [17]:
toy.pivot("id", "group") # Somewhat similar to .group with two columns, but now zeros are not omitted! 

group,1,2,3
A,1,0,2
B,1,3,0
C,1,0,1


Now, let's say another researcher gave us a new piece of information about the data:

In [8]:
toy_add = Table().with_columns("group", make_array("A","B","C"),
                               "info", make_array("control","treatment","placebo"))
toy_add

group,info
A,control
B,treatment
C,placebo


We don't want to store this additional information separately and want to just add it to the 'toy' data frame.

**Quesion 1.6.** How can we merge the two data tables?

In [9]:
toy = toy.join("group", toy_add)
toy

group,id,val_1,val_2,info
A,1,1,-0.535532,control
A,3,7,-0.659394,control
A,3,8,-2.7284,control
B,1,2,-0.347885,treatment
B,2,4,-0.911886,treatment
B,2,5,1.34077,treatment
B,2,6,0.0547437,treatment
C,1,3,0.345067,placebo
C,3,9,-1.34297,placebo


**Advice:** When you are changing the data you are working with (like in the cell above) it might be better to first assign the result to a new variable and then inspect it. Then if it looks right you can change the actual data set, otherwise you can debug your code. This approach will prevent you from needing to rerun all of the cells before the current one in case something goes wrong.  

## 2.Titanic data

The next example uses the Titanic data, located in the "data" folder.

In [10]:
titanic = Table.read_table("data/titanic.csv")
titanic.show(32)

Class,Sex,Age,Survived,Freq
1st,Male,Child,No,0
2nd,Male,Child,No,0
3rd,Male,Child,No,35
Crew,Male,Child,No,0
1st,Female,Child,No,0
2nd,Female,Child,No,0
3rd,Female,Child,No,17
Crew,Female,Child,No,0
1st,Male,Adult,No,118
2nd,Male,Adult,No,154


**Question 2.1.** Create a 2x2 table with 'Sex' as rows, 'Survived' as columns and containing the information about how many man and women survived and died.

The intuitive thing to do might be the following:

In [11]:
titanic.pivot("Survived", "Sex")

Sex,No,Yes
Female,8,8
Male,8,8


But note that this way we only count the number of rows in Titanic data that contain the corresponding information. Instead we need to sum the 'Freq' values as well. 

In [12]:
tb1 = titanic.pivot("Survived", "Sex", values="Freq", collect=sum)
tb1

Sex,No,Yes
Female,126,344
Male,1364,367


**Question 2.2.** To the table you created in the previous question add a column of total counts of male/female passengers. *Hint: use .apply()*

In [13]:
tb2 = tb1.with_columns("Total", tb1.drop('Sex').apply(sum))
tb2

Sex,No,Yes,Total
Female,126,344,470
Male,1364,367,1731


**Question 2.3.** Now do the same table for Adult vs Child instead of Male vs Female.

In [14]:
tb3 = titanic.pivot("Survived", "Age", values="Freq", collect=sum)
tb3 = tb3.with_columns("Total", tb3.drop('Age').apply(sum))
tb3

Age,No,Yes,Total
Adult,1438,654,2092
Child,52,57,109


**Question 2.4.** Can you think of a way to create a table like in Question 2.2 but with only adult male and female passengers?

In [15]:
tb4 = titanic.where('Age', 'Adult')
tb4 = tb4.pivot("Survived", "Sex", values="Freq", collect=sum)
tb4 = tb4.with_columns("Total", tb4.drop('Sex').apply(sum))
tb4

Sex,No,Yes,Total
Female,109,316,425
Male,1329,338,1667


Note that in the solution above we did each step separately, but we could also combine some of them together:

In [16]:
tb4 = titanic.where('Age', 'Adult').pivot("Survived", "Sex", values="Freq", collect=sum)
tb4 = tb4.with_columns("Total", tb4.drop('Sex').apply(sum))
tb4

Sex,No,Yes,Total
Female,109,316,425
Male,1329,338,1667
